# text mining (nlp) with python

**Author:** Ties de Kok ([Personal Website](http://www.tiesdekok.com))  
**Last updated:** 18 May 2018  
**Python version:** Python 3.6  
**License:** MIT License  

**Note:** Some features (like the ToC) will only work if you run the notebook or if you use nbviewer by clicking this link:  
https://nbviewer.jupyter.org/github/TiesdeKok/Python_NLP_Tutorial/blob/master/NLP_Notebook.ipynb

# *Introduction*

This notebook contains code examples to get you started with Natural Language Processing (NLP) / Text Mining for Research and Data Science purposes.  

In the large scheme of things there are roughly 4 steps:  

1. Identify a data source  
2. Gather the data  
3. Process the data  
4. Analyze the data  

This notebook only discusses step 3 and 4. If you want to learn more about step 2 see my [Python tutorial](https://github.com/TiesdeKok/LearnPythonforResearch). 

## Note: companion slides

This notebook was designed to accompany a PhD course session on NLP techniques in Accounting Research.  
The slides of this session are publically availabe here: [Slides](http://www.tiesdekok.com/AccountingNLP_Slides/)

# *Elements / topics that are discussed in this notebook: *


<img style="float: left" src="https://i.imgur.com/c3aCZLA.png" width="50%" /> 

# *Table of Contents*  <a id='toc'></a>

* [Primer on NLP tools](#tool_primer)     
* [Process + Clean text](#proc_clean)   
    * [Normalization](#normalization)
        * [Deal with unwanted characters](#unwanted_char)
        * [Sentence segmentation](#sentence_seg)   
        * [Word tokenization](#word_token)
        * [Lemmatization & Stemming](#lem_and_stem) 
    * [Language modeling](#lang_model) 
        * [Part-of-Speech tagging](#pos_tagging) 
        * [Uni-Gram & N-Grams](#n_grams) 
        * [Stop words](#stop_words) 
* [Direct feature extraction](#feature_extract) 
    * [Feature search](#feature_search) 
        * [Entity recognition](#entity_recognition) 
        * [Pattern search](#pattern_search) 
    * [Text evaluation](#text_eval) 
        * [Language](#language) 
        * [Dictionary counting](#dict_counting) 
        * [Readability](#readability) 
* [Represent text numerically](#text_numerical) 
    * [Bag of Words](#bows) 
        * [TF-IDF](#tfidf) 
    * [Word Embeddings](#word_embed) 
        * [Word2Vec](#Word2Vec) 
* [Statistical models](#stat_models) 
    * ["Traditional" machine learning](#trad_ml) 
        * [Supervised](#trad_ml_supervised) 
            * [Naïve Bayes](#trad_ml_supervised_nb) 
            * [Support Vector Machines (SVM)](#trad_ml_supervised_svm) 
        * [Unsupervised](#trad_ml_unsupervised) 
            * [Latent Dirichilet Allocation (LDA)](#trad_ml_unsupervised_lda) 
            * [pyLDAvis](#trad_ml_unsupervised_pyLDAvis) 
* [Model Selection and Evaluation](#trad_ml_eval) 
* [Neural Networks](#nn_ml)

# <span style="text-decoration: underline;">Primer on NLP tools</span><a id='tool_primer'></a> [(to top)](#toc)

There are many tools available for NLP purposes.  
The code examples below are based on what I personally like to use, it is not intended to be a comprehsnive overview.  

Besides build-in Python functionality I will use / demonstrate the following packages:

**Standard NLP libraries**:
1. `Spacy` and the higher-level wrapper `Textacy` 
2. `NLTK` and the higher-level wrapper `TextBlob`

*Note: besides installing the above packages you also often have to download (model) data . Make sure to check the documentation!*

**Standard machine learning library**:

1. `scikit learn`

**Specific task libraries**:

There are many, just a couple of examples:

1. `pyLDAvis` for visualizing LDA)
2. `langdetect` for detecting languages
3. `fuzzywuzzy` for fuzzy text matching
4. `textstat` to calculate readability statistics
5. `Gensim` for topic modelling

# <span style="text-decoration: underline;">Get some example data</span><a id='example_data'></a> [(to top)](#toc)

There are many example datasets available to play around with, see for example this great repository:  
https://archive.ics.uci.edu/ml/datasets.html?format=&task=&att=&area=&numAtt=&numIns=&type=text&sort=nameUp&view=table

The data that I will use for most of the examples is the "Reuter_50_50 Data Set" that is used for author identification experiments. 

See the details here: https://archive.ics.uci.edu/ml/datasets/Reuter_50_50  

### Download and load the data

Can't follow what I am doing here? Please see my [Python tutorial](https://github.com/TiesdeKok/LearnPythonforResearch) (although the `zipfile` and `io` operations are not very relevant).

In [2]:
import requests, zipfile, io, os

*Download and extract the zip file with the data *

In [3]:
if not os.path.exists('C50test'):
    r = requests.get("https://archive.ics.uci.edu/ml/machine-learning-databases/00217/C50.zip")
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()

*Load the data into memory*

In [4]:
folder_dict = {'test' : 'C50test'}
text_dict = {'test' : {}}

In [5]:
for label, folder in folder_dict.items():
    authors = os.listdir(folder)
    for author in authors:
        text_files = os.listdir(os.path.join(folder, author))
        for file in text_files:
            with open(os.path.join(folder, author, file), 'r') as text_file:
                text_dict[label].setdefault(author, []).append(' '.join(text_file.readlines()))

*Note: the text comes pre-split per sentence, for the sake of example I undo this through `' '.join(text_file.readlines()`*

In [6]:
text_dict['test']['TimFarrand'][0]

'Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain\'s Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts.\n Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers Commission which is due to report before March 24, 1997. The shares fell 6p to 781p on the news.\n "The stock is probably dead in the water until March," said John Wakley, analyst at Lehman Brothers.  \n Dermott Carr, an analyst at Nikko said, "the market is going to hang onto them for the moment but until we get a decision they will be held back."\n Whatever the MMC decides many analysts expect Lang to defer a decision until after the next general election which will be called by May 22.\n "They will probably try to defer the decision until after the election. I don\'t think they want the negative PR of having a large number of people fired," said Wakley.  \n If the deal does not go throu

# <span style="text-decoration: underline;">Process + Clean text</span><a id='proc_clean'></a> [(to top)](#toc)

## Convert the text into a NLP representation

We can use the text directly, but if want to use packages like `spacy` and `textblob` we first have to convert the text into a corresponding object.  

### Spacy

**Note:** depending on the way that you installed the language models you will need to import it differently:

```
from spacy.en import English
parser = English()
```
OR
```
import en_core_web_sm
parser = en_core_web_sm.load()
```

In [8]:
import en_core_web_sm
parser = en_core_web_sm.load()

Convert all text in the "test" sample to a `spacy` `doc` object using `parser()`:

In [9]:
spacy_text = {}
for author, text_list in text_dict['test'].items():
    spacy_text[author] = [parser(text) for text in text_list]

In [10]:
type(spacy_text['TimFarrand'][0])

spacy.tokens.doc.Doc

### NLTK

In [11]:
import nltk

We can apply basic `nltk` operations directly to the text so we don't need to convert first. 

### TextBlob

In [13]:
from textblob import TextBlob

Convert all text in the "test" sample to a `TextBlob` object using `TextBlob()`:

In [14]:
textblob_text = {}
for author, text_list in text_dict['test'].items():
    textblob_text[author] = [TextBlob(text) for text in text_list]

In [15]:
type(textblob_text['TimFarrand'][0])

textblob.blob.TextBlob

## <span style="text-decoration: underline;">Normalization</span><a id='normalization'></a> [(to top)](#toc)

**Text normalization** describes the task of transforming the text into a different (more comparable) form.  

This can imply many things, I will show a couple of things below:

### <span style="text-decoration: underline;">Deal with unwanted characters</span><a id='unwanted_char'></a> [(to top)](#toc)

You will often notice that there are characters that you don't want in your text.  

Let's look at this sentence for example:

> "Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain\'s Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts.\n Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers"

You notice that there are some `\` and `\n` in there. These are used to define how a string should be displayed, if we print this text we get:  

In [16]:
text_dict['test']['TimFarrand'][0][:298]

"Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain's Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts.\n Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers"

In [17]:
print(text_dict['test']['TimFarrand'][0][:298])

Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain's Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts.
 Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers


If we want to analyze text we often don't care about the visual representation. They might actually cause problems!  

** So how do we remove them? **

In many cases it is sufficient to simply use the `.replace()` function:

In [18]:
text_dict['test']['TimFarrand'][0][:298].replace('\n', '').replace('\\', '')

"Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain's Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts. Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers"

Sometimes, however, the problem arrises because of encoding / decoding problems.  

In those cases you can usually do something like:  

In [19]:
problem_sentence = 'This is some \\u03c0 text that has to be cleaned\\u2026! it\\u0027s annoying!'
print(problem_sentence.encode().decode('unicode_escape').encode('ascii','ignore'))

b"This is some  text that has to be cleaned! it's annoying!"


### <span style="text-decoration: underline;">Sentence segmentation</span><a id='sentence_seg'></a> [(to top)](#toc)

Sentence segmentation means the task of splitting up the piece of text by sentence.  

You could do this by splitting on the `.` symbol, but dots are used in many other cases as well so it is not very robust:

In [20]:
text_dict['test']['TimFarrand'][0][:550].split('.')

["Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain's Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts",
 '\n Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers Commission which is due to report before March 24, 1997',
 ' The shares fell 6p to 781p on the news',
 '\n "The stock is probably dead in the water until March," said John Wakley, analyst at Lehman Brothers',
 '  \n Dermott Carr, an analyst at Nikko said, "the mark']

It is better to use a more sophisticated implementation such as the one by `Spacy`:

In [21]:
example_paragraph = spacy_text['TimFarrand'][0]

In [22]:
sentence_list = [s for s in example_paragraph.sents]
sentence_list[:5]

[Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain's Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts.
  ,
 Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers Commission which is due to report before March 24, 1997.,
 The shares fell 6p to 781p on the news.
  ,
 "The stock is probably dead in the water until March," said John Wakley, analyst at Lehman Brothers.  
  ,
 Dermott Carr, an analyst at Nikko said, "the market is going to hang onto them for the moment but until we get a decision they will be held back."
  ]

Notice that the returned object is still a `spacy` object:

In [23]:
type(sentence_list[0])

spacy.tokens.span.Span

Apply to all texts (for use later on):

In [24]:
spacy_sentences = {}
for author, text_list in spacy_text.items():
    spacy_sentences[author] = [list(text.sents) for text in text_list]

In [25]:
spacy_sentences['TimFarrand'][0][:3]

[Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain's Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts.
  ,
 Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers Commission which is due to report before March 24, 1997.,
 The shares fell 6p to 781p on the news.
  ]

### <span style="text-decoration: underline;">Word tokenization</span><a id='word_token'></a> [(to top)](#toc)

Word tokenization means to split the sentence (or text) up into words.

In [26]:
example_sentence = spacy_sentences['TimFarrand'][0][0]
example_sentence

Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain's Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts.
 

A word is called a `token` in this context (hence `tokenization`), using `spacy`:

In [27]:
token_list = [token for token in example_sentence]
token_list[0:15]

[Shares,
 in,
 brewing,
 -,
 to,
 -,
 leisure,
 group,
 Bass,
 Plc,
 are,
 likely,
 to,
 be,
 held]

### <span style="text-decoration: underline;">Lemmatization & Stemming</span><a id='lem_and_stem'></a> [(to top)](#toc)

In some cases you want to convert a word (i.e. token) into a more general representation.  

For example: convert "car", "cars", "car's", "cars'" all into the word `car`.

This is generally done through lemmatization / stemming (different approaches trying to achieve a similar goal).  

**Spacy**

Space offers build-in functionality for lemmatization:

In [28]:
lemmatized = [token.lemma_ for token in example_sentence]
lemmatized[0:15]

['share',
 'in',
 'brewing',
 '-',
 'to',
 '-',
 'leisure',
 'group',
 'bass',
 'plc',
 'be',
 'likely',
 'to',
 'be',
 'hold']

**NLTK**

Using the NLTK libary we can also use the more aggressive Porter Stemmer

In [29]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [30]:
stemmed = [stemmer.stem(token.text) for token in example_sentence]
stemmed[0:15]

['share',
 'in',
 'brew',
 '-',
 'to',
 '-',
 'leisur',
 'group',
 'bass',
 'plc',
 'are',
 'like',
 'to',
 'be',
 'held']

**Compare**:

In [31]:
for original, lemma, stem in zip(token_list[:15], lemmatized[:15], stemmed[:15]):
    print(original, ' | ', lemma, ' | ', stem)

Shares  |  share  |  share
in  |  in  |  in
brewing  |  brewing  |  brew
-  |  -  |  -
to  |  to  |  to
-  |  -  |  -
leisure  |  leisure  |  leisur
group  |  group  |  group
Bass  |  bass  |  bass
Plc  |  plc  |  plc
are  |  be  |  are
likely  |  likely  |  like
to  |  to  |  to
be  |  be  |  be
held  |  hold  |  held


In my experience it is usually best to use lemmatization instead of a stemmer. 

## <span style="text-decoration: underline;">Language modeling</span><a id='lang_model'></a> [(to top)](#toc)

Text is inherently structured in complex ways, we can often use some of this underlying structure. 

### <span style="text-decoration: underline;">Part-of-Speech tagging</span><a id='pos_tagging'></a> [(to top)](#toc)

Part of speech tagging refers to the identification of words as nouns, verbs, adjectives, etc. 

Using `Spacy`:

In [32]:
pos_list = [(token, token.pos_) for token in example_sentence]
pos_list[0:10]

[(Shares, 'NOUN'),
 (in, 'ADP'),
 (brewing, 'NOUN'),
 (-, 'PUNCT'),
 (to, 'ADP'),
 (-, 'PUNCT'),
 (leisure, 'NOUN'),
 (group, 'NOUN'),
 (Bass, 'PROPN'),
 (Plc, 'PROPN')]

### <span style="text-decoration: underline;">Uni-Gram & N-Grams</span><a id='n_grams'></a> [(to top)](#toc)

Obviously a sentence is not a random collection of words, the sequence of words has information value.  

A simple way to incorporate some of this sequence is by using what is called `n-grams`.  
An `n-gram` is nothing more than a a combination of `N` words into one token (a uni-gram token is just one word).  

So we can convert `"Sentence about flying cars"` into a list of bigrams:

> Sentence-about, about-flying, flying-cars  

See my slide on N-Grams for a more comprehensive example: [click here](http://www.tiesdekok.com/AccountingNLP_Slides/#14)

Using `NLTK`:

In [33]:
bigram_list = ['-'.join(x) for x in nltk.bigrams([token.text for token in example_sentence])]
bigram_list[10:15]

['are-likely', 'likely-to', 'to-be', 'be-held', 'held-back']

### <span style="text-decoration: underline;">Stop words</span><a id='stop_words'></a> [(to top)](#toc)

Depending on what you are trying to do it is possible that there are many words that don't add any information value to the sentence.  

The primary example are stop words.  

Sometimes you can improve the accuracy of your model by removing stop words.

Using `Spacy`:

In [34]:
no_stop_words = [token for token in example_sentence if not token.is_stop]

In [35]:
no_stop_words[:10]

[Shares, brewing, -, -, leisure, group, Bass, Plc, likely, held]

In [36]:
token_list[:10]

[Shares, in, brewing, -, to, -, leisure, group, Bass, Plc]

*Note* we can also remove punctuation in the same way:

In [37]:
[token for token in example_sentence if not token.is_punct][:5]

[Shares, in, brewing, to, leisure]

## Wrap everything into one function

Below I will primarily use `SpaCy` directly. However, I also recommend to check out the high-level wrapper `Textacy`.

See their GitHub page for details: https://github.com/chartbeat-labs/textacy

### Quick `Textacy` example

In [38]:
import textacy

In [39]:
example_text = text_dict['test']['TimFarrand'][0]

In [40]:
cleaned_text = textacy.preprocess_text(example_text, lowercase=True, fix_unicode=True, no_punct=True)

** Basic SpaCy text processing function **

1. Split into sentences
2. Apply lemmatizer and remove top words
3. Clean up the sentence using `textacy`

In [41]:
def process_text_custom(text):
    sentences = list(parser(text).sents)
    lemmatized_sentences = []
    for sentence in sentences:
        lemmatized_sentences.append([token.lemma_ for token in sentence if not token.is_stop | token.is_punct | token.is_space])
    return [parser(' '.join(sentence)) for sentence in lemmatized_sentences]

In [42]:
%%time
spacy_text_clean = {}
for author, text_list in text_dict['test'].items():
    lst = []
    for text in text_list:
        lst.append(process_text_custom(text))
    spacy_text_clean[author] = lst

Wall time: 14min 45s


Note that there are quite a lot of sentences (~52K) so this takes a bit of time (~ 2 minutes).

In [43]:
count = 0
for author, texts in spacy_text_clean.items():
    for text in texts:
        count += len(text)
print('Number of sentences:', count)

Number of sentences: 56125


Result

In [44]:
spacy_text_clean['TimFarrand'][0][:3]

[share brewing leisure group bass plc likely hold britain 's trade industry secretary ian lang decide allow propose merge brewer carlsberg tetley say analyst,
 early lang announce bass deal refer monoplies mergers commission report march 24 1997,
 the share fall 6p 781p news]

# <span style="text-decoration: underline;">Direct feature extraction</span><a id='feature_extract'></a> [(to top)](#toc)

We now have pre-processed our text into something that we can use for direct feature extraction or to convert it to a numerical representation. 

## <span style="text-decoration: underline;">Feature search</span><a id='feature_search'></a> [(to top)](#toc)

### <span style="text-decoration: underline;">Entity recognition</span><a id='entity_recognition'></a> [(to top)](#toc)

It is often useful / relevant to extract entities that are mentioned in a piece of text.   

SpaCy is quite powerful in extracting entities, however, it doesn't work very well on lowercase text.  

Given that "token.lemma\_" removes capitalization I will use `spacy_sentences` for this example.

In [46]:
example_sentence = spacy_sentences['TimFarrand'][0][3]
example_sentence

"The stock is probably dead in the water until March," said John Wakley, analyst at Lehman Brothers.  
 

In [47]:
[(i, i.label_) for i in parser(example_sentence.text).ents]

[(March, 'DATE'), (John Wakley, 'PERSON'), (Lehman Brothers, 'ORG')]

In [48]:
example_sentence = spacy_sentences['TimFarrand'][4][0]
example_sentence

British pub-to-hotel group Greenalls Plc on Thursday reported a 48 percent rise in profits before exceptional items to 148.7 million pounds ($246.4 million), driven by its acquisition of brewer Boddington in November 1995.
 

In [49]:
[(i, i.label_) for i in parser(example_sentence.text).ents]

[(British, 'NORP'),
 (Greenalls Plc, 'PERSON'),
 (Thursday, 'DATE'),
 (48 percent, 'PERCENT'),
 (148.7 million pounds, 'MONEY'),
 ($246.4 million, 'MONEY'),
 (Boddington, 'GPE'),
 (November 1995, 'DATE')]

### <span style="text-decoration: underline;">Pattern search</span><a id='pattern_search'></a> [(to top)](#toc)

Using the build-in `re` (regular expression) library you can pattern match nearly anything you want.  

I will not go into details about regular expressions but see here for a tutorial:  
https://regexone.com/references/python  

In [50]:
import re

**TIP**: Use [Pythex.org](https://pythex.org/) to try out your regular expression

Example on Pythex: <a href="https://pythex.org/?regex=IDNUMBER: (\d\d\d-\w\w)&test_string=Ties de Kok (IDNUMBER: 123-AZ). Rest of Text." target='_blank'>click here</a>

**Example 1:**  

In [51]:
string_1 = 'Ties de Kok (#IDNUMBER: 123-AZ). Rest of text...'
string_2 = 'Philip Joos (#IDNUMBER: 663-BY). Rest of text...'

In [52]:
pattern = r'#IDNUMBER: (\d\d\d-\w\w)'

In [53]:
print(re.findall(pattern, string_1)[0])
print(re.findall(pattern, string_2)[0])

123-AZ
663-BY


### Example 2:

If a sentence contains the word 'million' return True, otherwise return False

In [54]:
for sen in spacy_text_clean['TimFarrand'][2]:
    TERM = 'million'
    contains = True if re.search('million', sen.text) else False
    if contains:
        print(sen)

analyst forecast pretax profit range 218 232 million stg restructuring cost 206 million time
a restructure cost 35 million anticipate bulk 25 million stem closure small production plant france
cadbury 's u.s. drink business turn 112 million stg trading profit 59 million half 1995 entirely contribution dr pepper
campbell estimate uk beverage contribute 47 million stg operating profit 50 million time
broadly analyst expect pretty flat performance group 's confectionery business consensus forecast 110 million stg operating profit
on average analyst calculate beverage chip trading profit 150 million
after sale 51 percent stake coca cola amp schweppes beverages ccsb operation coca cola enterprises june 620 million stg analyst want clear statement strategy company
but far analyst company say shareholder expect return investment emerge market large far 75 million russian plant
cadbury announce investment 20 million stg build new plant wrocoaw poland 1993 joint venture china cost 20 million
ne

## <span style="text-decoration: underline;">Text evaluation</span><a id='text_eval'></a> [(to top)](#toc)

Besides feature search there are also many ways to analyze the text as a whole.  

Let's, for example, evaluate the following paragraph:

In [55]:
example_paragraph = ' '.join([x.text for x in spacy_text_clean['TimFarrand'][2]])
example_paragraph[:500]

"soft drink confectionery group cadbury schweppes plc expect report solid percent rise half profit wednesday face question performance 7up soft drink u.s. one main question success relaunch 7up brand say mark duffy food manufacturing analyst sbc warburg competitor sprite own coca cola see agressive marketing push rank fast grow brand u.s. cadbury 's dr pepper analyst forecast pretax profit range 218 232 million stg restructuring cost 206 million time a dividend 5.1 penny expect 4.9p a restructure"

### <span style="text-decoration: underline;">Language</span><a id='language'></a> [(to top)](#toc)

Using the `langdetect` package it is easy to detect the language of a piece of text

In [57]:
from langdetect import detect

In [58]:
detect(example_paragraph)

'en'

### <span style="text-decoration: underline;">Readability</span><a id='readability'></a> [(to top)](#toc)

Using the `textstat` package we can compute various readability metrics

https://github.com/shivam5992/textstat

In [59]:
from textstat.textstat import textstat

In [60]:
print(textstat.flesch_reading_ease(example_paragraph))
print(textstat.smog_index(example_paragraph))
print(textstat.flesch_kincaid_grade(example_paragraph))
print(textstat.coleman_liau_index(example_paragraph))
print(textstat.automated_readability_index(example_paragraph))
print(textstat.dale_chall_readability_score(example_paragraph))
print(textstat.difficult_words(example_paragraph))
print(textstat.linsear_write_formula(example_paragraph))
print(textstat.gunning_fog(example_paragraph))
print(textstat.text_standard(example_paragraph))

21.2
18.9
20.5
16.67
26.0
9.09
89
8.6
27.120776699029125
8th and 9th grade


## Text similarity

In [61]:
from fuzzywuzzy import fuzz

In [62]:
fuzz.ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear")

91

### <span style="text-decoration: underline;">Term (dictionary) counting</span><a id='dict_counting'></a> [(to top)](#toc)

One of the most common techniques that researchers currently use (at least in Accounting research) are simple metrics based on counting words in a dictionary.  
This technique is, for example, very prevalent in sentiment analysis (counting positive and negative words).  

In essence this technique is very simple to program:

### Example 1:

In [63]:
word_dictionary = ['soft', 'first', 'most', 'be']

In [64]:
for word in word_dictionary:
    print(word, example_paragraph.count(word))

soft 3
first 0
most 1
be 8


### Example 2:

In [65]:
pos = ['great', 'increase']
neg = ['bad', 'decrease']

sentence = '''According to Trump everything is great, great, 
and great even though his popularity is seeing a decrease.'''

pos_count = 0
for word in pos:
    pos_count += sentence.lower().count(word)
print(pos_count)

neg_count = 0
for word in neg:
    neg_count += sentence.lower().count(word)
print(neg_count)

pos_count / (neg_count + pos_count)

3
1


0.75

In [66]:
sentence = '''According to Trump everything is great, great, 
and great even though his popularity is seeing a decrease.'''

In [67]:
pos_count = 0
for word in pos:
    pos_count += sentence.lower().count(word)
print(pos_count)

3


In [68]:
neg_count = 0
for word in neg:
    neg_count += sentence.lower().count(word)
print(neg_count)

1


In [69]:
pos_count / (neg_count + pos_count)

0.75

Getting the total number of words is also easy:

In [70]:
len(parser(example_paragraph))

419

#### Example 3:

We can also save the count per word

In [71]:
pos_count_dict = {}
for word in pos:
    pos_count_dict[word] = sentence.lower().count(word)

In [72]:
pos_count_dict

{'great': 3, 'increase': 0}

# <span style="text-decoration: underline;">Represent text numerically</span><a id='text_numerical'></a> [(to top)](#toc)

## <span style="text-decoration: underline;">Bag of Words</span><a id='bows'></a> [(to top)](#toc)

Sklearn includes the `CountVectorizer` and `TfidfVectorizer` function.  

For details, see the documentation:  
[TF](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer)  
[TFIDF](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer)

Note 1: these functions also already includes a lot of preprocessing options (e.g. ngrames, remove stop words, accent stripper).

Note 2: example based on the following website [click here](http://ethen8181.github.io/machine-learning/clustering_old/tf_idf/tf_idf.html)

In [73]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

### Simple example:

In [74]:
doc_1 = "The sky is blue."
doc_2 = "The sun is bright today."
doc_3 = "The sun in the sky is bright."
doc_4 = "We can see the shining sun, the bright sun."

Calculate term frequency:

In [75]:
vectorizer = CountVectorizer(stop_words='english')
tf = vectorizer.fit_transform([doc_1, doc_2, doc_3, doc_4])

In [76]:
print(vectorizer.get_feature_names())
for doc_tf_vector in tf.toarray():
    print(doc_tf_vector)

['blue', 'bright', 'shining', 'sky', 'sun', 'today']
[1 0 0 1 0 0]
[0 1 0 0 1 1]
[0 1 0 1 1 0]
[0 1 1 0 2 0]


### <span style="text-decoration: underline;">TF-IDF</span><a id='tfidf'></a> [(to top)](#toc)

In [77]:
transformer = TfidfVectorizer(stop_words='english')
tfidf = transformer.fit_transform([doc_1, doc_2, doc_3, doc_4])

In [78]:
for doc_vector in tfidf.toarray():
    print(doc_vector)

[0.78528828 0.         0.         0.6191303  0.         0.        ]
[0.         0.47380449 0.         0.         0.47380449 0.74230628]
[0.         0.53256952 0.         0.65782931 0.53256952 0.        ]
[0.         0.36626037 0.57381765 0.         0.73252075 0.        ]


### More elaborate example:

In [79]:
clean_paragraphs = []
for author, value in spacy_text_clean.items():
    for article in value:
        clean_paragraphs.append(' '.join([x.text for x in article]))

In [80]:
len(clean_paragraphs)

2500

In [81]:
transformer = TfidfVectorizer(stop_words='english')
tfidf_large = transformer.fit_transform(clean_paragraphs)

In [82]:
print('Number of vectors:', len(tfidf_large.toarray()))
print('Number of words in dictionary:', len(tfidf_large.toarray()[0]))

Number of vectors: 2500
Number of words in dictionary: 24092


In [83]:
tfidf_large

<2500x24092 sparse matrix of type '<class 'numpy.float64'>'
	with 446676 stored elements in Compressed Sparse Row format>

## <span style="text-decoration: underline;">Word Embeddings</span><a id='word_embed'></a> [(to top)](#toc)

### <span style="text-decoration: underline;">Word2Vec</span><a id='Word2Vec'></a> [(to top)](#toc)

Simple example below is from:  https://medium.com/@mishra.thedeepak/word2vec-in-minutes-gensim-nlp-python-6940f4e00980

In [79]:
import gensim
from nltk.corpus import brown

In [80]:
sentences = brown.sents()
model = gensim.models.Word2Vec(sentences, min_count=1)

Save model

In [81]:
model.save('brown_model')

Load model

In [82]:
model = gensim.models.Word2Vec.load('brown_model')

Find words most similar to 'mother':

In [83]:
print(model.most_similar("mother"))

[('father', 0.9751995801925659), ('husband', 0.9580737352371216), ('wife', 0.9531918168067932), ('son', 0.9327206611633301), ('voice', 0.9201934337615967), ('boy', 0.9148358106613159), ('friend', 0.9091534614562988), ('ache', 0.8969202041625977), ('parents', 0.8916678428649902), ('maid', 0.8907431960105896)]


Find the odd one out:

In [84]:
print(model.doesnt_match("breakfast cereal dinner lunch".split()))

cereal


In [85]:
print(model.doesnt_match("pizza pasta garden fries".split()))

garden


Retrieve vector representation of the word "human"

In [86]:
model['human']

array([ -1.24233282e+00,   1.73863158e-01,  -6.12607360e-01,
        -5.31224608e-01,   2.60317028e-01,  -3.56246114e-01,
        -5.06218493e-01,  -1.15960002e-01,   3.04902345e-01,
        -1.21463396e-01,   4.93332326e-01,  -9.75235820e-01,
         3.44474047e-01,   9.77401040e-04,   5.05028106e-02,
        -3.87262404e-01,   4.93362129e-01,   7.00087488e-01,
        -6.27336025e-01,  -4.63026613e-01,   2.79739406e-02,
         1.45691419e+00,   4.07162786e-01,  -4.19379741e-01,
        -8.41612220e-01,   8.46711546e-02,   3.03834379e-01,
        -5.89724183e-01,  -5.50288737e-01,   1.14675418e-01,
        -9.28169414e-02,  -5.00818849e-01,   1.74140222e-02,
         2.45587930e-01,   9.37732458e-02,  -6.30766377e-02,
         3.79322648e-01,  -9.30945396e-01,   1.81099728e-01,
         4.46529061e-01,   5.09826422e-01,  -4.00113940e-01,
        -3.06686193e-01,   5.83700202e-02,  -1.30845475e+00,
        -8.19562197e-01,  -1.43999264e-01,  -1.79302439e-01,
        -9.88642037e-01,

# <span style="text-decoration: underline;">Statistical models</span><a id='stat_models'></a> [(to top)](#toc)

## <span style="text-decoration: underline;">"Traditional" machine learning</span><a id='trad_ml'></a> [(to top)](#toc)

The library to use for machine learning is scikit-learn (["sklearn"](http://scikit-learn.org/stable/index.html)).

## <span>Supervised</span><a id='trad_ml_supervised'></a> [(to top)](#toc)

In [84]:
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.externals import joblib

In [85]:
import pandas as pd
import numpy as np

### Convert the data into a pandas dataframe (so that we can input it easier)

In [86]:
article_list = []
for author, value in spacy_text_clean.items():
    for article in value:
        article_list.append((author, ' '.join([x.text for x in article])))

In [87]:
article_df = pd.DataFrame(article_list, columns=['author', 'text'])

In [88]:
article_df.sample(5)

,author,text
384,DavidLawder,chrysler corp. report record earning 1996 base...
377,DavidLawder,tease automotive world glimpse possible produc...
2114,SarahDavison,a widespread shake hit highly competitive fund...
737,JanLopatka,engineering group skoda a.s say tuesday win or...
2070,SamuelPerry,intel corp. executive say late tuesday company...


### Split the sample into a training and test sample

In [89]:
X_train, X_test, y_train, y_test = train_test_split(article_df.text, article_df.author, test_size=0.20, random_state=3561)

In [90]:
print(len(X_train), len(X_test))

2000 500


### Train and evaluate function

Simple function to train (i.e. fit) and evaluate the model

In [91]:
def train_and_evaluate(clf, X_train, X_test, y_train, y_test):
    
    clf.fit(X_train, y_train)
    
    print("Accuracy on training set:")
    print(clf.score(X_train, y_train))
    print("Accuracy on testing set:")
    print(clf.score(X_test, y_test))
    
    y_pred = clf.predict(X_test)
    
    print("Classification Report:")
    print(metrics.classification_report(y_test, y_pred))

### <span>Naïve Bayes estimator</span><a id='trad_ml_supervised_nb'></a> [(to top)](#toc)

In [92]:
from sklearn.naive_bayes import MultinomialNB

Define pipeline

In [93]:
clf = Pipeline([
    ('vect', TfidfVectorizer(strip_accents='unicode',
                             lowercase = True,
                            max_features = 1500,
                            stop_words='english'
                            )),
        
    ('clf', MultinomialNB(alpha = 1,
                          fit_prior = True
                          )
    ),
])

Train and show evaluation stats

In [94]:
train_and_evaluate(clf, X_train, X_test, y_train, y_test)

Accuracy on training set:
0.8345
Accuracy on testing set:
0.714
Classification Report:
                   precision    recall  f1-score   support

    AaronPressman       0.90      1.00      0.95         9
       AlanCrosby       0.55      0.92      0.69        12
   AlexanderSmith       0.86      0.60      0.71        10
  BenjaminKangLim       0.75      0.27      0.40        11
    BernardHickey       0.75      0.30      0.43        10
      BradDorfman       0.80      1.00      0.89         8
 DarrenSchuettler       0.58      0.78      0.67         9
      DavidLawder       1.00      0.60      0.75        10
    EdnaFernandes       1.00      0.67      0.80         9
      EricAuchard       0.86      0.67      0.75         9
   FumikoFujisaki       1.00      1.00      1.00        10
   GrahamEarnshaw       0.59      1.00      0.74        10
 HeatherScoffield       0.83      0.56      0.67         9
       JanLopatka       0.33      0.33      0.33         9
    JaneMacartney       0.3

Save results

In [95]:
joblib.dump(clf, 'naive_bayes_results.pkl')

['naive_bayes_results.pkl']

Predict out of sample:

In [96]:
example_y, example_X = y_train[33], X_train[33]

In [97]:
print('Actual author:', example_y)
print('Predicted author:', clf.predict([example_X])[0])

Actual author: AaronPressman
Predicted author: AaronPressman


### <span>Support Vector Machines (SVM)</span><a id='trad_ml_supervised_svm'></a> [(to top)](#toc)

In [98]:
from sklearn.svm import SVC

Define pipeline

In [99]:
clf_svm = Pipeline([
    ('vect', TfidfVectorizer(strip_accents='unicode',
                             lowercase = True,
                            max_features = 1500,
                            stop_words='english'
                            )),
        
    ('clf', SVC(kernel='rbf' ,
                C=10, gamma=0.3)
    ),
])

*Note:* The SVC estimator is very sensitive to the hyperparameters!

Train and show evaluation stats

In [100]:
train_and_evaluate(clf_svm, X_train, X_test, y_train, y_test)

Accuracy on training set:
0.9965
Accuracy on testing set:
0.83
Classification Report:
                   precision    recall  f1-score   support

    AaronPressman       0.89      0.89      0.89         9
       AlanCrosby       0.79      0.92      0.85        12
   AlexanderSmith       1.00      0.70      0.82        10
  BenjaminKangLim       0.67      0.36      0.47        11
    BernardHickey       1.00      0.50      0.67        10
      BradDorfman       0.70      0.88      0.78         8
 DarrenSchuettler       1.00      0.89      0.94         9
      DavidLawder       1.00      0.70      0.82        10
    EdnaFernandes       0.73      0.89      0.80         9
      EricAuchard       0.69      1.00      0.82         9
   FumikoFujisaki       1.00      1.00      1.00        10
   GrahamEarnshaw       0.77      1.00      0.87        10
 HeatherScoffield       0.75      1.00      0.86         9
       JanLopatka       0.43      0.33      0.38         9
    JaneMacartney       0.36

Save results

In [101]:
joblib.dump(clf_svm, 'svm_results.pkl')

['svm_results.pkl']

Predict out of sample:

In [102]:
example_y, example_X = y_train[33], X_train[33]

In [103]:
print('Actual author:', example_y)
print('Predicted author:', clf_svm.predict([example_X])[0])

Actual author: AaronPressman
Predicted author: AaronPressman


## <span>Model Selection and Evaluation</span><a id='trad_ml_eval'></a> [(to top)](#toc)

Both the `TfidfVectorizer` and `SVC()` estimator take a lot of hyperparameters.  

It can be difficult to figure out what the best parameters are.

We can use `GridSearchCV` to help figure this out.

In [104]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score

First we define the options that should be tried out:

In [105]:
clf_search = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', SVC())
])
parameters = { 'vect__stop_words': ['english'],
                'vect__strip_accents': ['unicode'],
              'vect__max_features' : [1500],
              'vect__ngram_range': [(1,1), (2,2) ],
             'clf__gamma' : [0.2, 0.3, 0.4], 
             'clf__C' : [8, 10, 12],
              'clf__kernel' : ['rbf']
             }

Run everything:

In [106]:
grid = GridSearchCV(clf_search, param_grid=parameters, scoring=make_scorer(f1_score, average='micro'), n_jobs=1)
grid.fit(X_train, y_train)    

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__stop_words': ['english'], 'vect__strip_accents': ['unicode'], 'vect__max_features': [1500], 'vect__ngram_range': [(1, 1), (2, 2)], 'clf__gamma': [0.2, 0.3, 0.4], 'clf__C': [8, 10, 12], 'clf__kernel': ['rbf']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(f1_score, average=micro), verbose=0)

*Note:* if you are on a powerful unix system you can set n_jobs to the number of available threads to speed up the calculation

In [107]:
print("The best parameters are %s with a score of %0.2f" % (grid.best_params_, grid.best_score_))
y_true, y_pred = y_test, grid.predict(X_test)
print(metrics.classification_report(y_true, y_pred))

The best parameters are {'clf__C': 12, 'clf__gamma': 0.4, 'clf__kernel': 'rbf', 'vect__max_features': 1500, 'vect__ngram_range': (1, 1), 'vect__stop_words': 'english', 'vect__strip_accents': 'unicode'} with a score of 0.77
                   precision    recall  f1-score   support

    AaronPressman       0.89      0.89      0.89         9
       AlanCrosby       0.79      0.92      0.85        12
   AlexanderSmith       1.00      0.70      0.82        10
  BenjaminKangLim       0.67      0.36      0.47        11
    BernardHickey       1.00      0.50      0.67        10
      BradDorfman       0.70      0.88      0.78         8
 DarrenSchuettler       1.00      0.89      0.94         9
      DavidLawder       1.00      0.70      0.82        10
    EdnaFernandes       0.80      0.89      0.84         9
      EricAuchard       0.75      1.00      0.86         9
   FumikoFujisaki       1.00      1.00      1.00        10
   GrahamEarnshaw       0.77      1.00      0.87        10
 HeatherS

## <span>Unsupervised</span><a id='trad_ml_unsupervised'></a> [(to top)](#toc)

### <span>Latent Dirichilet Allocation (LDA)</span><a id='trad_ml_unsupervised_lda'></a> [(to top)](#toc)

In [108]:
from sklearn.decomposition import LatentDirichletAllocation

Vectorizer (using countvectorizer for the sake of example)

In [109]:
vectorizer = CountVectorizer(strip_accents='unicode',
                             lowercase = True,
                            max_features = 1500,
                            stop_words='english', max_df=0.8)
tf_large = vectorizer.fit_transform(clean_paragraphs)

Run the LDA model

In [110]:
n_topics = 10
n_top_words = 25

In [111]:
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=10,
                                learning_method='online',
                                n_jobs=1)
lda_fitted = lda.fit_transform(tf_large)

Visualize top words

In [112]:
def save_top_words(model, feature_names, n_top_words):
    out_list = []
    for topic_idx, topic in enumerate(model.components_):
        out_list.append((topic_idx+1, " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])))
    out_df = pd.DataFrame(out_list, columns=['topic_id', 'top_words'])
    return out_df

In [113]:
result_df = save_top_words(lda, vectorizer.get_feature_names(), n_top_words)

In [114]:
result_df

,topic_id,top_words
0,1,company service new corp computer internet net...
1,2,percent million analyst share profit quarter s...
2,3,bank financial market company banking business...
3,4,tonne 000 ford new production plant car gm exp...
4,5,company pound million share group percent bill...
5,6,thomson nomura conrail csf north south korean ...
6,7,percent market price czech oil state billion t...
7,8,government state boeing plan tobacco union new...
8,9,stock gold bre company share canada toronto ex...
9,10,china kong hong chinese beijing people deng of...


### <span>pyLDAvis</span><a id='trad_ml_unsupervised_pyLDAvis'></a> [(to top)](#toc)

In [116]:
%matplotlib inline
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

C:\Users\kokti\Anaconda3\lib\site-packages\sklearn\manifold\t_sne.py:420: DeprecationWarning: invalid escape sequence \s
  """


In [117]:
pyLDAvis.sklearn.prepare(lda, tf_large, vectorizer, n_jobs=1)

C:\Users\kokti\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      16.969439        1       1 -0.213244 -0.037263
4      15.968033        1       2 -0.066392 -0.003642
9      13.834050        1       3  0.175962  0.085620
0      10.946849        1       4 -0.050486  0.102120
2      10.098617        1       5 -0.010880 -0.005670
7       8.618809        1       6  0.164622  0.083481
3       7.239122        1       7 -0.027935  0.142638
6       7.141099        1       8 -0.066723 -0.041015
8       5.592496        1       9 -0.068922 -0.098116
5       3.591488        1      10  0.164000 -0.228153, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
301   Default  3469.000000          china  3469.000000  30.0000  30.0000
777   Default  2356.000000           kong  2356.000000  29.0000  29.0000
686   Default  2343.000000           hong  2343.000000  28.0000  28.0000
196   Default  2560.000000           bank  2560.000000  27.0000  27.0000
1316  Default  2146.000000          stock  2146.000000  26.0000  26.0000
997   Default  5573.000000        percent  5573.000000  25.0000  25.0000
1242  Default  3429.000000          share  3429.000000  24.0000  24.0000
209   Default  1370.000000        beijing  1370.000000  23.0000  23.0000
302   Default  1564.000000        chinese  1564.000000  22.0000  22.0000
1387  Default   935.000000          tonne   935.000000  21.0000  21.0000
1104  Default  1497.000000        quarter  1497.000000  20.0000  20.0000
1035  Default  1563.000000          pound  1563.000000  19.0000  19.0000
344   Default  5507.000000        company  5507.000000  18.0000  18.0000
1072  Default  1877.000000         profit  1877.000000  17.0000  17.0000
854   Default  4213.000000         market  4213.000000  16.0000  16.0000
643   Default  2023.000000     government  2023.000000  15.0000  15.0000
1376  Default   555.000000        thomson   555.000000  14.0000  14.0000
1234  Default  1520.000000        service  1520.000000  13.0000  13.0000
353   Default   850.000000       computer   850.000000  12.0000  12.0000
1204  Default  2136.000000           sale  2136.000000  11.0000  11.0000
641   Default   597.000000           gold   597.000000  10.0000  10.0000
1056  Default  2098.000000          price  2098.000000   9.0000   9.0000
233   Default   558.000000            bre   558.000000   8.0000   8.0000
0     Default  1599.000000            000  1599.000000   7.0000   7.0000
740   Default   698.000000       internet   698.000000   6.0000   6.0000
439   Default   780.000000           deng   780.000000   5.0000   5.0000
884   Default  5035.000000        million  5035.000000   4.0000   4.0000
1306  Default  1542.000000          state  1542.000000   3.0000   3.0000
552   Default   760.000000         export   760.000000   2.0000   2.0000
218   Default  3292.000000        billion  3292.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
1228  Topic10   168.655248          seoul   171.907391   3.3075  -4.6695
134   Topic10   175.498022        alcatel   180.974578   3.2959  -4.6297
1211  Topic10   154.160125        scandal   165.478824   3.2558  -4.7594
745   Topic10   200.387034  investigation   215.951842   3.2518  -4.4971
1353  Topic10   188.761680        suspend   213.264518   3.2046  -4.5569
1063  Topic10   101.500301          probe   119.978790   3.1594  -5.1773
778   Topic10   217.007254          korea   257.521255   3.1554  -4.4174
507   Topic10    81.172785        embassy   100.926818   3.1088  -5.4008
744   Topic10    84.938931    investigate   111.278596   3.0565  -5.3554
1459  Topic10    60.926611       watchdog    97.778037   2.8536  -5.6877
628   Topic10    59.150920            gec   100.120705   2.8003  -5.7173
758   Topic10   173.651234       japanese   309.754994   2.7479  -4.6403
449   Topic10    50.7836

## <span style="text-decoration: underline;">Neural Networks</span><a id='nn_ml'></a> [(to top)](#toc)

Interested? Check out the Stanford course CS224n ([Syllabus](http://web.stanford.edu/class/cs224n/syllabus.html))!   